<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-805abf20-292d-f672-ebcb-e3ed354478a6)


# Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py

--2025-04-04 13:27:49--  https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10129 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.89K  --.-KB/s    in 0s      

2025-04-04 13:27:50 (85.9 MB/s) - ‘helper_functions.py’ saved [10129/10129]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

## Get a text dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2025-04-04 13:27:53--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.003s  

2025-04-04 13:27:53 (166 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing data
with the help of pandas


In [5]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# suffling train data
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [8]:
# how many samples?
len(train_df),len(test_df)

(7613, 3263)

In [9]:
# Lets visualize some random samples.
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","real disaster" if target>0 else "not real disaster")
  print(f"Text: {text}")
  print("_____")

Target:0 not real disaster
Text: Devia ler 'AS WE COLLIDE #wattys2015' no #Wattpad #teenfiction http://t.co/g891m9GH4r http://t.co/Xq92X4bVG3
_____
Target:1 real disaster
Text: @SweetieBirks @mirrorlady2 @SLATUKIP So name all the countries we've invaded/bombed aside Libya in North Africa in last 5 years.....
_____
Target:1 real disaster
Text: #WorldNews
 VIDEO: 'We're picking up bodies from water' - BBC News - Home:
Rescuers are searching for hundreds of.. http://t.co/6hhmBdK9Yo
_____
Target:0 not real disaster
Text: @ATL_Events and thunder :-o
_____
Target:0 not real disaster
Text: Reddit's new content policy goes into effect many horrible subreddits banned or quarantined http://t.co/M4TcZaawpT
_____


In [10]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


## split data in to training and validation set

In [11]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1, # use 10% of training data for validation
                                                                       random_state=42)

In [12]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text vectorizarion (Tokenization)
### Converting text into numbers

*   Tokenization: Tokenization is the process of breaking down text into smaller units called tokens, which can be words, subwords, or even characters. Its a direct mapping of token to a number
*   Embedding: Create a matrix of feature vector for each token.
Embeddings convert tokens (words/subwords) into numerical vectors so that machines can process them. They capture semantic meaning (e.g., "king" is closer to "queen" than to "apple").Contextual Embeddings (Modern NLP)

Words get different vectors based on context (e.g., BERT, GPT).



In [14]:
import tensorflow as tf
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=None,
                                                  standardize='lower_and_strip_punctuation',
                                                  split='whitespace',
                                                  ngrams=None, #create groups of n-words
                                                  output_mode='int',
                                                  output_sequence_length=None, # how long do u want your sequence to be?
                                                  pad_to_max_tokens=False
                                                  )


### find the average number of tokens or words in training samples

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# setup text vectorization variables
max_vocab_length=10000 # Maximum number of words to have in our vocabolary
max_length=15 #max length our sentences will be
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                  output_mode='int',
                                                  output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# lets create a sample sentences and tokenize it
sample_sentence="The recent earthquake has destroyed the entire city"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 2756,  290,   41,  351,    2,  855,  182,    0,    0,    0,
           0,    0,    0,    0]])>

In [19]:
# selecting a random sample from the train dataset and tokenize it
import random
random_sentence=random.choice(train_sentences)
print(f"Random Sample: {random_sentence}")
print(f"Vectorized sample: {text_vectorizer([random_sentence])}")

Random Sample: yhngsjlg just tweeted about their secret crush:http://t.co/IoqM5bm1Dg
Vectorized sample: [[6560   29 7292   54  131 1060    1    0    0    0    0    0    0    0
     0]]


In [20]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # Get all the unique words in the training dataset
top_5_words=words_in_vocab[:5] # get the top 5 most common words
bottom_5_words=words_in_vocab[-5:] # get the least 5 common words
print(f"Words in Vocab: {len(words_in_vocab)}")
print(f"Top 5 words: {top_5_words}")
print(f"Least 5 words: {bottom_5_words}")

Words in Vocab: 10000
Top 5 words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Least 5 words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


### creating an Embedding using Embedding layer

*   input_dim=the size of our vocabolary
*   output_dim= the size of the output embedding vector
*   input_length= size of the sequences being passed to the embedding layer

Key Properties of an Embedding Layer

**Trainable:**

* The vectors are updated via backpropagation (like other weights in a neural network).

* Pretrained embeddings (e.g., GloVe) can also be fine-tuned.

**Efficiency:**
More efficient than one-hot encoding + dense layers (avokes high-dimensional sparse matrices).

**Interpretability:**
Learned embeddings often capture semantic relationships (e.g., similar words cluster together).


In [21]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length, # set input shape
                           output_dim=128, # output shape
                           input_length=max_length # how long is each input
                           )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
# embed the random sample. i.e turn into dense vector of fixed size
print("Random sample: ",random_sentence)
print("After Embedding:...................")
sample_embed=embedding(text_vectorizer([random_sentence]))
sample_embed

Random sample:  yhngsjlg just tweeted about their secret crush:http://t.co/IoqM5bm1Dg
After Embedding:...................


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04750867, -0.01634829,  0.01217735, ..., -0.03293909,
         -0.04437698, -0.00347301],
        [-0.00107812,  0.00235659, -0.00624417, ..., -0.04342412,
         -0.03407084, -0.02666506],
        [-0.03759283, -0.01787513, -0.03138872, ..., -0.01359341,
         -0.02532905,  0.03438525],
        ...,
        [-0.04075596, -0.02694322, -0.01149314, ...,  0.02356544,
         -0.01788811, -0.0184713 ],
        [-0.04075596, -0.02694322, -0.01149314, ...,  0.02356544,
         -0.01788811, -0.0184713 ],
        [-0.04075596, -0.02694322, -0.01149314, ...,  0.02356544,
         -0.01788811, -0.0184713 ]]], dtype=float32)>

In [23]:
# checkout single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04750867, -0.01634829,  0.01217735, -0.0205727 , -0.02155693,
         0.02227386,  0.0425815 ,  0.04474722,  0.00212876, -0.02037611,
         0.01406601, -0.01707289,  0.03006861, -0.0108547 ,  0.00513178,
        -0.03296848,  0.00586813, -0.00908231,  0.0141163 ,  0.03043605,
        -0.04673674,  0.01955315,  0.03703674,  0.03349702, -0.00452993,
         0.04898577,  0.01230222, -0.03578608, -0.04350333, -0.0026885 ,
         0.02602745,  0.01578338, -0.03582063, -0.03223892, -0.01126931,
         0.01218548, -0.0087014 , -0.0418889 ,  0.02323773, -0.00551235,
        -0.0449059 ,  0.00261838, -0.00990706,  0.02705488, -0.04723687,
        -0.00551463, -0.0315782 , -0.01049148,  0.03090061,  0.0064312 ,
         0.04776896, -0.03810191, -0.01664967,  0.004451  , -0.04113122,
        -0.04907739, -0.0097171 ,  0.04492887, -0.04998432, -0.04446477,
        -0.0163942 , -0.02970066, -0.00510741, -0.0053255 ,  0.03654094,
  

This is the embedding for the first word of sentence

# Model 0: creating a baseline
To create baseline, let's use Sklearn Multimodal Naive Bayes using TF-IDF formula to convert text into numbers.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# create tokenization and modeling pipeline
model_0=Pipeline([ # pipeline is like keras Sequantial here
    ('tfidf',TfidfVectorizer()), # convert words into numbers using tfidf
    ('clf',MultinomialNB()), #model the text
])
# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

## Evaluate baseline model

In [25]:
baseline_accuracy=model_0.score(val_sentences,val_labels)
print(f"Model 0 (Baseline) Accuracy: {baseline_accuracy*100:.3f} %")

Model 0 (Baseline) Accuracy: 79.265 %


In [26]:
baseline_preds=model_0.predict(val_sentences)
print("Predicted labels")
baseline_preds[:20]

Predicted labels


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
print("Actual labels")
val_labels[:20]

Actual labels


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [28]:
def evaluate_model(model,true_labels,predicted_labels):
  precision=tf.keras.metrics.Precision()
  precision.update_state(true_labels,predicted_labels)
  recall=tf.keras.metrics.Recall()
  recall.update_state(true_labels,predicted_labels)
  f1=(2*(precision.result()*recall.result()))/(precision.result()+recall.result())
  print(f"Precision: {precision.result()*100:.2f}%")
  print(f"Recall: {recall.result()*100:.2f}%")
  print(f"F1 score: {f1*100:.2f}%")

evaluate_model(model_0,val_labels,baseline_preds)



Precision: 88.62%
Recall: 62.64%
F1 score: 73.40%


In [29]:
# from sk-learn
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_performance_metrics(true_labels,predicted_labels):
  accuracy=accuracy_score(true_labels,predicted_labels)
  precision_score,recall_score,f1_score,_=precision_recall_fscore_support(true_labels,predicted_labels,average="weighted")
  model_results={
      "accuracy":accuracy,
      "precision":precision_score,
      "recall":recall_score,
      "f1_score":f1_score
  }
  return model_results

In [58]:
model_0_performance=calculate_performance_metrics(val_labels,baseline_preds)
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

The two evaluation methods (evaluate_model and calculate_performance_metrics) produce different results because:
* Different Libraries & Implementations

tf.keras.metrics (TensorFlow) and sklearn.metrics (scikit-learn) compute metrics differently.

Example:

>tf.keras.metrics.Precision() uses micro-averaging by default.

>sklearn.metrics.precision_recall_fscore_support() uses weighted averaging (as specified by average="weighted").
Averaging Method

* Weighted vs. Micro Averaging:

>Weighted (scikit-learn): Accounts for class imbalance by weighting metrics by class support.

>Micro (TensorFlow): Aggregates all classes equally, which can skew results if classes are imbalanced.

# Model 1: Simple dense model

In [32]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback
# create a directory to save the tensorboard logs
SAVE_DIR="mdoel_logs"

In [41]:
# Building a model with functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string") # inputs are one dimentional string
x=text_vectorizer(inputs) # turn input text into number
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # lower the dimentionality of the embedding
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")

In [42]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 1)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ text_vectorization_1                 │ (None, 15)                  │               0 │
│ (TextVectorization)                  │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding (Embedding)                │ (None, 15, 128)             │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                          callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_1_dense")])


Saving TensorBoard log files to: mdoel_logs/model_1_dense/20250404-142228
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.6429 - loss: 0.6469 - val_accuracy: 0.7520 - val_loss: 0.5368
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8100 - loss: 0.4665 - val_accuracy: 0.7848 - val_loss: 0.4735
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8640 - loss: 0.3510 - val_accuracy: 0.7848 - val_loss: 0.4601
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8921 - loss: 0.2831 - val_accuracy: 0.7887 - val_loss: 0.4668
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9086 - loss: 0.2472 - val_accuracy: 0.7913 - val_loss: 0.4882


In [44]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7818 - loss: 0.5224


[0.48823294043540955, 0.7913385629653931]

In [46]:
# make predictions
model_1_pred_probability=model_1.predict(val_sentences)
model_1_pred_probability.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


(762, 1)

In [47]:
model_1_pred_probability[0]

array([0.3380792], dtype=float32)

In [53]:
model_1_pred_probability=tf.squeeze(tf.round(model_1_pred_probability))
model_1_pred_probability[:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.],
      dtype=float32)>

In [54]:
val_labels[:50]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0])

In [62]:
model_1_perfornamce=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_1_pred_probability)
model_1_perfornamce

{'accuracy': 0.7913385826771654,
 'precision': 0.7986313296680336,
 'recall': 0.7913385826771654,
 'f1_score': 0.7877263030663991}

In [63]:
# compare with the baseline
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}